# Load dependencies

In [81]:
import cobra
import libsbml
import lxml

models_directory = '/media/sf_Shared/Systems_biology/Metabolic_models/'
OB3b_directory = '/media/sf_Shared/GEM_OB3b/'
memote_directory = '/home/ensakz/Desktop/memote_m_trichosporium/'
fastas_directory = '/media/sf_Shared/Systems_biology/Fastas_and_annotations/'
inparanoid_directory = '/media/sf_Shared/Systems_biology/InParanoid_runs/'
draft_gems_dir_8 = 'draft_gems/8.OB3b_draft_gem_during_pathway_manual_curation/'
gem_confirmation = 'gem_confirmation/'

In [82]:
iMsOB3b = cobra.io.load_json_model(models_directory + "iMsOB3b.json")

In [83]:
iMcBath = cobra.io.load_json_model(models_directory + "iMcBath.json")

In [84]:
universal = cobra.io.load_json_model(models_directory + "universal_model.json")

# Add heterologous reactions for cadaverine production
## Add Pyruvate carboxylase (pyc) to iMsOB3b:

In [85]:
iMsOB3b.add_reaction(iMcBath.reactions.PC)

In [86]:
iMsOB3b.reactions.PC

Reaction identifier,PC
Name,Pyruvate carboxylase
Memory address,0x07f46e5faa5f8
Stoichiometry,atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c ATP + HCO3_ + pyruvate --> ADP + H_ + oxaloacetate + phosphate
GPR,MCA2479 or MCA2480 or MCA2481
Lower bound,0.0
Upper bound,1000.0


## Add lysine decarboxylase (ldcC/cadA) to iMsOB3b:

In [87]:
iMsOB3b.add_reaction(universal.reactions.LYSDC)

In [88]:
iMsOB3b.reactions.LYSDC

Reaction identifier,LYSDC
Name,Lysine decarboxylase
Memory address,0x07f46ca66f710
Stoichiometry,"h_c + lys__L_c --> 15dap_c + co2_c H_ + L-Lysine --> 1,5-Diaminopentane + CO2"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [89]:
iMsOB3b.reactions.LYSDC.reaction = "h_c + lys_L_c --> 15dap_c + co2_c"

## Add cadaverine-lysine antiporter protein to iMsOB3b:

In [90]:
iMsOB3b.add_reaction(universal.reactions.CADVt)

In [91]:
iMsOB3b.reactions.CADVt

Reaction identifier,CADVt
Name,Lysine/Cadaverine antiporter
Memory address,0x07f46d549f160
Stoichiometry,"15dap_c + h_e + lys__L_e --> 15dap_e + h_c + lys__L_c 1,5-Diaminopentane + H_ + L-Lysine --> 1,5-Diaminopentane + H_ + L-Lysine"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [92]:
iMsOB3b.reactions.CADVt.reaction = "15dap_c + h_e + lys_L_e --> 15dap_e + h_c + lys_L_c"

unknown metabolite 'lys_L_e' created


Check whether iMsOB3b hsa lys_L_e:

In [93]:
iMsOB3b.metabolites.lys_L_e

Metabolite identifier,lys_L_e
Name,
Memory address,0x07f46e26ceb70
Formula,None
Compartment,None
In 1 reaction(s),CADVt


## Add lysine demand reaction
This is required due to cadaverine-lysine antiporter protein reaction, i.e. iMsOB3b does not have external lysine metabolite.
<br>**Note:** I had a problem with using model.summary() function (I got error) when I added reaction with "EX_lys_L_e" to iMsOB3b model. Therefore, I renamed lysine reaction to "DM_lys_L_e" to get rid of the error.

In [95]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_lys_L_e')
reaction.name = 'Demand lysine'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [96]:
iMsOB3b.add_reaction(reaction)

In [97]:
iMsOB3b.reactions.get_by_id('DM_lys_L_e').reaction = 'lys_L_e -->'

In [98]:
iMsOB3b.reactions.get_by_id('DM_lys_L_e').reaction = '--> lys_L_e'

In [99]:
iMsOB3b.metabolites.lys_L_e.compartment = "e"

## Add cadaverine exchange reaction:

In [104]:
iMsOB3b.add_reaction(universal.reactions.EX_15dap_e)

Fix reaction annotation in iMsOB3b:

In [106]:
for reaction in iMsOB3b.reactions:
    reaction.annotation = {}

In [107]:
iMsOB3b.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  26.4   Biomass_Mext...  0.123
ch4_e      14.9       co2_e   9.97
h_e         5.36
no3_e       1.2
pi_e        0.0857
k_e         0.0226
so4_e       0.00774
mg2_e       0.00101
fe2_e       0.000984
ca2_e       0.000607
cl_e        0.000607
na1_e       0.000499
mobd_e      0.000432
cu2_e       0.000405
mn2_e       0.000405
zn2_e       0.000405
cobalt2_e   2.69e-05


Save altered model before simulations:

In [108]:
cobra.io.save_json_model(iMsOB3b, "/media/sf_Shared/Projects/Cadaverine_production/Models/" + 
                         "iMsOB3b_cadaverine.json")